<a href="https://colab.research.google.com/github/rmhanna/Randa/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import re

# 1. Load dataset
# Ensure 'train.csv' is uploaded to your Colab environment
df = pd.read_csv(r"C:\Users\randa\OneDrive\Documents\anaconda_projects\train.csv")

# --- Step 1: Remove blank category_name rows ---
blank_mask = df["category_name"].isna() | (df["category_name"].astype(str).str.strip() == "")
rows_blank = int(blank_mask.sum())   # <-- note this number for your report
df = df[~blank_mask]

# --- Step 2: Split category_name into multiple columns ---
split_cols = df["category_name"].str.split("/", n=-1, expand=True)
num_created = split_cols.shape[1]    # <-- note this number for your report

for i in range(num_created):
    df[f"category_name_{i+1}"] = split_cols[i]

# Remove new columns that have missing values in any row
for i in range(num_created):
    col = f"category_name_{i+1}"
    if df[col].isna().any():
        df.drop(columns=[col], inplace=True)

# --- Step 3: Normalize 'name' column (approximate clustering) ---
def normalize_name(s):
    if pd.isna(s):
        return s
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)          # collapse spaces
    s = re.sub(r"[^\w\s&-]", "", s)    # remove punctuation except &, -
    return s

df["name_norm"] = df["name"].apply(normalize_name)
num_clusters = df["name_norm"].nunique()   # <-- note this number for your report

# --- Step 4: Replace nulls in brand_name with "Unbranded" ---
df["brand_name"] = df["brand_name"].fillna("Unbranded")

# --- Step 5: Create high_priced column ---
df["high_priced"] = (df["price"] > 100).astype(int)

# --- Step 6: Create has_offer column ---
kw = ["discount", "offer", "sale"]
desc = df["item_description"].astype(str).str.lower()
df["has_offer"] = desc.apply(lambda x: int(any(k in x for k in kw)))

# --- Save cleaned dataset ---
df.to_csv("mercari_cleaned.csv", index=False)

# --- Quick summary outputs ---
print("Rows removed (blank category_name):", rows_blank)
print("Number of category_name columns created:", num_created)
print("Number of clusters (approx normalized names):", num_clusters)
print("Cleaned dataset saved as mercari_cleaned.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\randa\\OneDrive\\Documents\\anaconda_projects\\train.csv'